# Algorithm comparation & research

In [14]:
pip install niapy scipy

   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ----- ---------------------------------- 5.5/38.5 MB 40.1 MB/s eta 0:00:01
   ----------------- ---------------------- 16.5/38.5 MB 47.9 MB/s eta 0:00:01
   ----------------------- ---------------- 23.1/38.5 MB 42.6 MB/s eta 0:00:01
   ------------------------------- -------- 30.4/38.5 MB 40.5 MB/s eta 0:00:01
   --------------------------------- ------ 32.2/38.5 MB 34.1 MB/s eta 0:00:01
   ---------------------------------------  38.3/38.5 MB 33.9 MB/s eta 0:00:01
   ---------------------------------------- 38.5/38.5 MB 31.8 MB/s  0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from niapy.task import Task
from niapy.algorithms.basic import (
    FireflyAlgorithm,
    ParticleSwarmOptimization,
    GreyWolfOptimizer
)
from niapy.problems import Sphere, Rastrigin, Rosenbrock

## Benchmark functions

### Sphere

\[
    f(x) = \sum^{D}_{i=1} x^2_i
\]

* Unimodal
* Convex
* Single global minimum at x = 0
* Smooth gradient, easy convergence

### Rastrigin

\[
    f(x) = 10D + \sum^{D}_{i=1} (x^2_i - 10cos(2\pi x_i))
\]

* Multimodal
* Non-convex
* Many local minima and global minimum at x = 0
* Difficult convergence and challenging for premature convergence

### Rosenbrock

\[
    f(x) = \sum^{D-1}_{i=1} (100(x_{i+1} - x^2_i)^2 + (x_i - 1)^2)
\]

* Unimodal
* Non-convex
* Global minimum at x = (1, 1, 1, \dots, 1)
* Narrow curved valley

### Function parameters

In [2]:
problems = {
    "Sphere": (Sphere, {
        "lower": -5.12,
        "upper": 5.12,
        "max_evals": 10000,
    }),
    "Rastrigin": (Rastrigin, {
        "lower": -5.12,
        "upper": 5.12,
        "max_evals": 30000,
    }),
    "Rosenbrock": (Rosenbrock, {
        "lower": -30,
        "upper": 30,
        "max_evals": 30000,
    }),
}

> **Note:** `niapy` already initializes algorithms with a uniform random distribution by default over the upper and lower bounds defined, so there is no need to apply transformations to the functions due to centering-around-zero bias.

## Algorithms

### Firefly Algorithm

[Yang *et al*., 2009](https://link.springer.com/chapter/10.1007/978-3-642-04944-6_14)

### Particle Swarm Optimization

[Kennedy and Everhart, 1995](https://ieeexplore.ieee.org/document/488968)

### Grey Wolf Optimizer

[Mirjalili *et al*., 2014](https://www.sciencedirect.com/science/article/pii/S0965997813001853?casa_token=Wm5JRdpOZrUAAAAA:Z8DWX5cN2vzQPpJ-9mOd-yUu7GkgJM1Lb_gA1owhmITuD3HKMZZyf3jZ75q43UfmEDc9MWu6)

## Benchmarking

In [3]:
RESULTS_DIR = "results"
N_RUNS = 30
POPULATION_SIZE = 30

In [4]:
algorithms = {
    "FA": (FireflyAlgorithm, {
        "population_size": 30,
        "alpha": 0.8,
        "beta0": 1.0,
        "gamma": 0.1
    }),
    "PSO": (ParticleSwarmOptimization, { # Clerc's constriction coefficients
        "population_size": 30,
        "w": 0.729,
        "c1": 2.05,
        "c2": 2.05
    }),
    "GWO": (GreyWolfOptimizer, {
        "population_size": 30
    }),
}

In [5]:
def run_single(task, algorithm):
    """
    Run a single instance of an algorithm on a Task.

    Parameters
    ----------
    task : Task
        The optimization task to solve.
    algorithm : Algorithm
        The algorithm instance to run.

    Returns
    -------
    best_position : np.ndarray
        The best solution found.
    best_fitness : float
        The fitness value of the best solution.
    """
    best_position, best_fitness = algorithm.run(task)
    return best_position, best_fitness

In [6]:
import numpy as np

def benchmark_algorithm(algorithm_class, algorithm_params, problem_class, problem_params, n_runs=30, n_points=300, seed_start=0):
    """
    Runs an algorithm multiple times on a problem and collect results.

    Returns
    -------
    fitness_runs : np.ndarray
        Best fitness from each run (length n_runs)
    convergence_avg : np.ndarray
        Averaged convergence curve interpolated onto a common x-axis (length n_points)
    """
    fitness_runs = []
    convergence_curves = []

    evals_max = problem_params["max_evals"]
    x_common = np.linspace(0, evals_max, n_points)

    for run_idx in range(seed_start, seed_start + n_runs):
        np.random.seed(run_idx)

        # Create task and algorithm
        task = Task(
            problem_class(**problem_params),
            max_evals=problem_params["max_evals"],
            max_iters=problem_params["max_evals"] // algorithm_params["population_size"]
        )
        algo = algorithm_class(**algorithm_params)

        # Run single optimization
        best_position, best_fitness = algo.run(task)
        fitness_runs.append(best_fitness)

        # Get convergence data (function evaluations vs best fitness)
        x, y = task.convergence_data(x_axis="evals")

        # Interpolate onto common x-axis
        y_interp = np.interp(x_common, x, y)
        convergence_curves.append(y_interp)

    # Average convergence across runs
    convergence_avg = np.mean(convergence_curves, axis=0)

    return np.array(fitness_runs), convergence_avg

In [7]:
def run_benchmark_loop(problems, algorithms, *, dimension=30, n_runs=30, results_dir="results", n_points=300, exp_name="benchmark"):
    """
    Runs benchmark for multiple algorithms on multiple problems.
    Saves plots and summary table.
    """

    plots_dir = os.path.join(results_dir, f"{exp_name}/plots")
    tables_dir = os.path.join(results_dir, f"{exp_name}/tables")
    os.makedirs(plots_dir, exist_ok=True)
    os.makedirs(tables_dir, exist_ok=True)

    results = {}

    summary_rows = []

    for problem_name, (pclass, pconf) in problems.items():
        plt.figure()
        for algo_name, (algo_class, algo_params) in algorithms.items():
            # Combine dimension with problem config
            problem_params = {"dimension": dimension, **pconf}

            # Run benchmark
            fitness_runs, convergence_avg = benchmark_algorithm(
                algo_class,
                algo_params,
                problem_class=pclass,
                problem_params=problem_params,
                n_runs=n_runs,
                n_points=n_points
            )

            if problem_name not in results:
                results[problem_name] = {}
            results[problem_name][algo_name] = fitness_runs

            # Save summary statistics
            summary_rows.append({
                "Problem": problem_name,
                "Algorithm": algo_name,
                "Mean fitness": np.mean(fitness_runs),
                "Std fitness": np.std(fitness_runs),
                "Best fitness": np.min(fitness_runs),
                "Worst fitness": np.max(fitness_runs),
            })

            # Plot convergence
            plt.plot(np.linspace(0, problem_params["max_evals"], n_points), convergence_avg, label=algo_name)

        plt.title(f"{problem_name} – Convergence")
        plt.xlabel("Function evaluations")
        plt.ylabel("Best fitness")
        plt.yscale("log")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(plots_dir, f"{problem_name}_convergence.png"))
        plt.close()

    # Save summary table
    df_summary = pd.DataFrame(summary_rows)
    df_summary.to_csv(os.path.join(tables_dir, "summary.csv"), index=False)

    return results

In [8]:
res_5d = run_benchmark_loop(problems, algorithms, dimension=5, exp_name="5d")

In [9]:
res_30d = run_benchmark_loop(problems, algorithms, dimension=30, exp_name="30d")

### Statistic Tests

In [10]:
import numpy as np
from scipy.stats import friedmanchisquare, rankdata

def friedman_test(results):
    """
    Perform Friedman test on the results.

    Parameters
    ----------
    results : dict
        Dictionary with structure results[problem_name][algorithm_name] = list of fitness values

    Returns
    -------
    stat : float
        Friedman statistic
    p_value : float
        p-value of the test
    """

    algorithms = results[list(results.keys())[0]].keys()
    problems = results.keys()

    mean_matrix = np.array([
        [np.mean(results[p][a]) for a in algorithms]
        for p in problems
    ])

    stat, p_value = friedmanchisquare(
        mean_matrix[:, 0],
        mean_matrix[:, 1],
        mean_matrix[:, 2]
    )

    print(f"Friedman statistic = {stat:.4f}")
    print(f"p-value = {p_value:.4e}")

    ranks = np.array([
        rankdata(row, method="average")
        for row in mean_matrix
    ])

    avg_ranks = np.mean(ranks, axis=0)

    print("\nAverage ranks\n----------------")
    for algo, r in zip(algorithms, avg_ranks):
        print(f"{algo}: average rank = {r:.3f}")

    return stat, p_value

In [11]:
ft_5d = friedman_test(res_5d)

Friedman statistic = 4.6667
p-value = 9.6972e-02

Average ranks
----------------
FA: average rank = 2.333
PSO: average rank = 2.667
GWO: average rank = 1.000


In [12]:
ft_30d = friedman_test(res_30d)

Friedman statistic = 6.0000
p-value = 4.9787e-02

Average ranks
----------------
FA: average rank = 2.000
PSO: average rank = 3.000
GWO: average rank = 1.000


In [13]:
from scipy.stats import wilcoxon
from itertools import combinations
import pandas as pd

def wilcoxon_test(results):

    algorithms = list(results[list(results.keys())[0]].keys())
    problems = results.keys()

    mean_matrix = np.array([
        [np.mean(results[p][a]) for a in algorithms]
        for p in problems
    ])

    alpha = 0.05
    n_comparisons = len(algorithms) * (len(algorithms) - 1) // 2
    alpha_bonf = alpha / n_comparisons

    print(f"Bonferroni-corrected alpha = {alpha_bonf:.4f}\n")

    df = pd.DataFrame(columns=["Algorithm 1", "Algorithm 2", "Statistic", "p-value", "Significant"])

    for i, j in combinations(range(len(algorithms)), 2):
        stat, p = wilcoxon(
            mean_matrix[:, i],
            mean_matrix[:, j],
            alternative="two-sided"
        )
        significant = p < alpha_bonf

        df.loc[len(df)] = [
            algorithms[i],
            algorithms[j],
            f"{stat:.3f}",
            f"{p:.4e}",
            significant
        ]

    return df

In [14]:
wilcoxon_5d = wilcoxon_test(res_5d)
wilcoxon_5d

Bonferroni-corrected alpha = 0.0167



,Algorithm 1,Algorithm 2,Statistic,p-value,Significant
0,FA,PSO,1.000,5.0000e-01,False
1,FA,GWO,0.000,2.5000e-01,False
2,PSO,GWO,0.000,2.5000e-01,False


In [15]:
wilcoxon_30d = wilcoxon_test(res_30d)
wilcoxon_30d

Bonferroni-corrected alpha = 0.0167



,Algorithm 1,Algorithm 2,Statistic,p-value,Significant
0,FA,PSO,0.000,2.5000e-01,False
1,FA,GWO,0.000,2.5000e-01,False
2,PSO,GWO,0.000,2.5000e-01,False


## Benchmarking with parameter optimization

In [16]:
import random

def sample_params(search_space):
    params = {}
    for k, v in search_space.items():
        if isinstance(v, tuple):
            params[k] = random.uniform(*v)
        else:
            params[k] = random.choice(v)
    return params

Tuning:

* Train: Sphere and Rastrigin
* Test: Rosenbrock

In [17]:
tuning_problems = [
    (Sphere, {
        "dimension": 30,
        "lower": -5.12,
        "upper": 5.12,
        "max_evals": 10_000,
    }),
    (Rastrigin, {
        "dimension": 30,
        "lower": -5.12,
        "upper": 5.12,
        "max_evals": 30_000,
    }),
]

In [18]:
test_problems = {
    "Rosenbrock": (Rosenbrock, {
        "lower": -30,
        "upper": 30,
        "max_evals": 30000,
    }),
}

In [19]:
def evaluate_algorithm(
    algo_class,
    algo_params,
    problem_class,
    problem_params,
    n_runs=10
):
    fitness = []
    for seed in range(n_runs):
        np.random.seed(seed)
        task = Task(
            problem_class(**problem_params),
            max_evals=problem_params["max_evals"],
            max_iters=problem_params["max_evals"] // algo_params["population_size"]
        )
        algo = algo_class(**algo_params)
        _, best = algo.run(task)
        fitness.append(best)
    return np.mean(fitness)

In [20]:
def random_search_tuning(
    algo_class,
    base_params,
    search_space,
    problems,
    n_trials=30
):
    best_score = float("inf")
    best_params = None

    for trial in range(n_trials):
        sampled = sample_params(search_space)
        algo_params = {**base_params, **sampled}

        scores = []
        for pclass, pparams in problems:
            score = evaluate_algorithm(
                algo_class,
                algo_params,
                pclass,
                pparams
            )
            scores.append(score)

        mean_score = np.mean(scores)

        if mean_score < best_score:
            best_score = mean_score
            best_params = algo_params

        print(
            f"Trial {trial+1}/{n_trials} | "
            f"Score: {mean_score:.3e}"
        )

    return best_params, best_score

### PSO tuning

In [21]:
PSO_SEARCH_SPACE = {
    "population_size": [20, 30, 40, 50],
    "c1": (1.0, 2.5),
    "c2": (1.0, 2.5),
}

In [22]:
best_pso_params, best_pso_score = random_search_tuning(
    ParticleSwarmOptimization,
    base_params={},
    search_space=PSO_SEARCH_SPACE,
    problems=tuning_problems,
    n_trials=30
)

print("Best PSO params:", best_pso_params)

Trial 1/30 | Score: 2.215e+02
Trial 2/30 | Score: 2.815e+02
Trial 3/30 | Score: 3.055e+02
Trial 4/30 | Score: 2.455e+02
Trial 5/30 | Score: 2.181e+02
Trial 6/30 | Score: 2.517e+02
Trial 7/30 | Score: 2.933e+02
Trial 8/30 | Score: 2.274e+02
Trial 9/30 | Score: 2.980e+02
Trial 10/30 | Score: 2.058e+02
Trial 11/30 | Score: 2.256e+02
Trial 12/30 | Score: 2.810e+02
Trial 13/30 | Score: 1.930e+02
Trial 14/30 | Score: 2.824e+02
Trial 15/30 | Score: 2.482e+02
Trial 16/30 | Score: 2.705e+02
Trial 17/30 | Score: 2.917e+02
Trial 18/30 | Score: 2.726e+02
Trial 19/30 | Score: 2.890e+02
Trial 20/30 | Score: 2.893e+02
Trial 21/30 | Score: 3.050e+02
Trial 22/30 | Score: 2.015e+02
Trial 23/30 | Score: 2.037e+02
Trial 24/30 | Score: 2.774e+02
Trial 25/30 | Score: 2.423e+02
Trial 26/30 | Score: 2.141e+02
Trial 27/30 | Score: 2.871e+02
Trial 28/30 | Score: 3.076e+02
Trial 29/30 | Score: 1.824e+02
Trial 30/30 | Score: 2.880e+02
Best PSO params: {'population_size': 50, 'c1': 1.4732247832994418, 'c2': 1.1185

In [23]:
pso_algorithms = {
    "PSO": (ParticleSwarmOptimization, {
        "population_size": 30,
        "w": 0.729,
        "c1": 2.05,
        "c2": 2.05
    }),
    "Tuned PSO": (ParticleSwarmOptimization, {
        **best_pso_params,
        "w": 0.729,
    }),
}

In [24]:
pso_tuned_results = run_benchmark_loop(test_problems, pso_algorithms, dimension=30, exp_name="pso_tuned")

### FA tuning

In [25]:
FA_SEARCH_SPACE = {
    "population_size": [20, 30, 40, 50],
    "alpha": (0.1, 0.9),
    "beta0": (0.5, 2.0),
    "gamma": (0.1, 5.0),
}

In [26]:
best_fa_params, best_fa_score = random_search_tuning(
    FireflyAlgorithm,
    base_params={},
    search_space=FA_SEARCH_SPACE,
    problems=tuning_problems,
    n_trials=30
)

print("Best FA params:", best_fa_params)

Trial 1/30 | Score: 2.608e+02
Trial 2/30 | Score: 2.561e+02
Trial 3/30 | Score: 2.656e+02
Trial 4/30 | Score: 2.485e+02
Trial 5/30 | Score: 2.476e+02
Trial 6/30 | Score: 2.460e+02
Trial 7/30 | Score: 2.582e+02
Trial 8/30 | Score: 2.617e+02
Trial 9/30 | Score: 2.423e+02
Trial 10/30 | Score: 2.670e+02
Trial 11/30 | Score: 2.499e+02
Trial 12/30 | Score: 2.601e+02
Trial 13/30 | Score: 2.451e+02
Trial 14/30 | Score: 2.430e+02
Trial 15/30 | Score: 2.498e+02
Trial 16/30 | Score: 2.461e+02
Trial 17/30 | Score: 2.519e+02
Trial 18/30 | Score: 2.616e+02
Trial 19/30 | Score: 2.473e+02
Trial 20/30 | Score: 2.447e+02
Trial 21/30 | Score: 2.573e+02
Trial 22/30 | Score: 2.455e+02
Trial 23/30 | Score: 2.583e+02
Trial 24/30 | Score: 2.552e+02
Trial 25/30 | Score: 2.472e+02
Trial 26/30 | Score: 2.418e+02
Trial 27/30 | Score: 2.504e+02
Trial 28/30 | Score: 2.580e+02
Trial 29/30 | Score: 2.524e+02
Trial 30/30 | Score: 2.404e+02
Best FA params: {'population_size': 30, 'alpha': 0.22911362671778177, 'beta0': 

In [27]:
fa_algorithms = {
    "FA": (FireflyAlgorithm, {
        "population_size": 30,
        "alpha": 0.8,
        "beta0": 1.0,
        "gamma": 0.1
    }),
    "Tuned FA": (FireflyAlgorithm, {
        **best_fa_params,
    }),
}

In [28]:
fa_tuned_results = run_benchmark_loop(test_problems, fa_algorithms, dimension=30, exp_name="fa_tuned")

### Tuned PSO vs Tuned FA vs GWO

In [29]:
tuned_algorithms = {
    "Tuned FA": (FireflyAlgorithm, {
        **best_fa_params
    }),
    "Tuned PSO": (ParticleSwarmOptimization, {
        **best_pso_params,
        "w": 0.729,
    }),
    "GWO": (GreyWolfOptimizer, {
        "population_size": 30
    }),
}

In [30]:
last_tuned_results = run_benchmark_loop(problems, tuned_algorithms, dimension=30, exp_name="last_tuned")

In [31]:
tuned_friedman = friedman_test(last_tuned_results)

Friedman statistic = 6.0000
p-value = 4.9787e-02

Average ranks
----------------
Tuned FA: average rank = 3.000
Tuned PSO: average rank = 2.000
GWO: average rank = 1.000


In [32]:
wilcoxon_df = wilcoxon_test(last_tuned_results)
wilcoxon_df

Bonferroni-corrected alpha = 0.0167



,Algorithm 1,Algorithm 2,Statistic,p-value,Significant
0,Tuned FA,Tuned PSO,0.000,2.5000e-01,False
1,Tuned FA,GWO,0.000,2.5000e-01,False
2,Tuned PSO,GWO,0.000,2.5000e-01,False
